In [122]:
import kagglehub
import os 
import pandas as pd
import numpy as np

# Download latest version
path = kagglehub.dataset_download("heesoo37/120-years-of-olympic-history-athletes-and-results")

csv_path = f"{path}/athlete_events.csv"

df = pd.read_csv(csv_path , sep=',' ,encoding="ISO-8859-1")

df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


## Data Cleaning

In [123]:
df.shape

(271116, 15)

In [124]:
df.columns

Index(['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team', 'NOC', 'Games',
       'Year', 'Season', 'City', 'Sport', 'Event', 'Medal'],
      dtype='object')

In [125]:
df.describe()

,ID,Age,Height,Weight,Year
count,271116.000000,261642.000000,210945.000000,208241.000000,271116.000000
mean,68248.954396,25.556898,175.338970,70.702393,1978.378480
std,39022.286345,6.393561,10.518462,14.348020,29.877632
min,1.000000,10.000000,127.000000,25.000000,1896.000000
25%,34643.000000,21.000000,168.000000,60.000000,1960.000000
50%,68205.000000,24.000000,175.000000,70.000000,1988.000000
75%,102097.250000,28.000000,183.000000,79.000000,2002.000000
max,135571.000000,97.000000,226.000000,214.000000,2016.000000


In [126]:
missing_val_percentage = (df.isnull().sum()/len(df) )*100
missing_val_percentage.sort_values(ascending=False)

Medal     85.326207
Weight    23.191180
Height    22.193821
Age        3.494445
Sex        0.000000
ID         0.000000
Name       0.000000
Team       0.000000
NOC        0.000000
Year       0.000000
Games      0.000000
Season     0.000000
City       0.000000
Sport      0.000000
Event      0.000000
dtype: float64

In [127]:
df_copy = df.copy()
df_copy['Medal'] = df_copy['Medal'].fillna('No Medal')
cols = ['Age' , 'Weight' , 'Height']
for col in cols:
    df_copy[col] = df_copy[col].fillna(df.groupby('Sport')[col].transform('median'))
    df_copy[col] = df_copy[col].fillna(df.groupby('Sex')[col].transform('median'))
    
print(df_copy[df_copy['Weight'].isnull()]['Sport'].unique())

[]


In [128]:
from sklearn.preprocessing import OneHotEncoder

In [129]:
df_copy.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,No Medal
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,No Medal
2,3,Gunnar Nielsen Aaby,M,24.0,175.0,71.0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,No Medal
3,4,Edgar Lindenau Aabye,M,34.0,182.0,95.0,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,No Medal


In [130]:
# one hot encode medal colums
encoded = pd.get_dummies(df_copy['Medal'], dtype=int)
df_copy = pd.concat([df_copy , encoded] , axis=1)
df_copy.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Bronze,Gold,No Medal,Silver
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,No Medal,0,0,1,0
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,No Medal,0,0,1,0
2,3,Gunnar Nielsen Aaby,M,24.0,175.0,71.0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,No Medal,0,0,1,0
3,4,Edgar Lindenau Aabye,M,34.0,182.0,95.0,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,0,1,0,0
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,No Medal,0,0,1,0


In [131]:
df_copy.sample(10)
df_copy.drop(columns=['ID'] , inplace=True)

In [132]:
# remove whitespaces
for col in df_copy.columns:
    df_copy.replace(" " , "")
df_copy.sample(10)

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Bronze,Gold,No Medal,Silver
47386,Juan Esteban Curuchet,M,23.0,175.0,65.0,Argentina,ARG,1988 Summer,1988,Summer,Seoul,Cycling,Cycling Men's Points Race,No Medal,0,0,1,0
93428,Raimo Yrj Heinonen,M,25.0,174.0,63.5,Finland,FIN,1960 Summer,1960,Summer,Roma,Gymnastics,Gymnastics Men's Parallel Bars,No Medal,0,0,1,0
25275,Walter Boninsegni,M,34.0,167.0,74.0,Italy,ITA,1936 Summer,1936,Summer,Berlin,Shooting,"Shooting Men's Rapid-Fire Pistol, 25 metres",No Medal,0,0,1,0
242570,Francis Tournefier,M,24.0,182.0,100.0,France,FRA,1988 Summer,1988,Summer,Seoul,Weightlifting,Weightlifting Men's Heavyweight I,No Medal,0,0,1,0
178985,Martin Oten,M,18.0,180.0,68.0,Slovakia,SVK,2006 Winter,2006,Winter,Torino,Cross Country Skiing,Cross Country Skiing Men's Sprint,No Medal,0,0,1,0
145021,Cheryl Maas,F,29.0,174.0,66.0,Netherlands,NED,2014 Winter,2014,Winter,Sochi,Snowboarding,Snowboarding Women's Slopestyle,No Medal,0,0,1,0
18721,Naoufel Ben Rabah,M,22.0,170.0,60.0,Tunisia,TUN,2000 Summer,2000,Summer,Sydney,Boxing,Boxing Men's Lightweight,No Medal,0,0,1,0
86934,Bo Henning Gustafsson,M,33.0,175.0,64.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Athletics,Athletics Men's 50 kilometres Walk,No Medal,0,0,1,0
221710,Helga Mara Sista de Salvateli,F,20.0,171.0,61.0,Argentina,ARG,1968 Winter,1968,Winter,Grenoble,Alpine Skiing,Alpine Skiing Women's Giant Slalom,No Medal,0,0,1,0
235234,Asako Takakura,F,28.0,163.0,50.0,Japan,JPN,1996 Summer,1996,Summer,Atlanta,Football,Football Women's Football,No Medal,0,0,1,0


In [133]:
duplicate_val_cols = ['Team' , 'Event' , 'Medal' , 'Year' , 'City' ,'Sport' , 'Games'] 
df_copy.drop_duplicates(subset=duplicate_val_cols , inplace=True)
df_copy.head()

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Bronze,Gold,No Medal,Silver
0,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,No Medal,0,0,1,0
1,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,No Medal,0,0,1,0
2,Gunnar Nielsen Aaby,M,24.0,175.0,71.0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,No Medal,0,0,1,0
3,Edgar Lindenau Aabye,M,34.0,182.0,95.0,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,0,1,0,0
4,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,No Medal,0,0,1,0


In [134]:
# we will only analyze by summer olympics
df_copy = df_copy[df_copy['Season']=='Summer']
df_copy.head()

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Bronze,Gold,No Medal,Silver
0,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,No Medal,0,0,1,0
1,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,No Medal,0,0,1,0
2,Gunnar Nielsen Aaby,M,24.0,175.0,71.0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,No Medal,0,0,1,0
3,Edgar Lindenau Aabye,M,34.0,182.0,95.0,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,0,1,0,0
26,"Cornelia ""Cor"" Aalten (-Strannood)",F,18.0,168.0,67.0,Netherlands,NED,1932 Summer,1932,Summer,Los Angeles,Athletics,Athletics Women's 100 metres,No Medal,0,0,1,0


In [135]:
df_copy.isnull().sum()

Name        0
Sex         0
Age         0
Height      0
Weight      0
Team        0
NOC         0
Games       0
Year        0
Season      0
City        0
Sport       0
Event       0
Medal       0
Bronze      0
Gold        0
No Medal    0
Silver      0
dtype: int64

In [136]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 105600 entries, 0 to 271099
Data columns (total 18 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Name      105600 non-null  object 
 1   Sex       105600 non-null  object 
 2   Age       105600 non-null  float64
 3   Height    105600 non-null  float64
 4   Weight    105600 non-null  float64
 5   Team      105600 non-null  object 
 6   NOC       105600 non-null  object 
 7   Games     105600 non-null  object 
 8   Year      105600 non-null  int64  
 9   Season    105600 non-null  object 
 10  City      105600 non-null  object 
 11  Sport     105600 non-null  object 
 12  Event     105600 non-null  object 
 13  Medal     105600 non-null  object 
 14  Bronze    105600 non-null  int64  
 15  Gold      105600 non-null  int64  
 16  No Medal  105600 non-null  int64  
 17  Silver    105600 non-null  int64  
dtypes: float64(3), int64(5), object(10)
memory usage: 15.3+ MB


## EDA

## Total Medals By each Country

In [ ]:
# get medals by each country
medal_tally = df_copy.groupby(['NOC' , 'Team']).sum()[['Gold' , 'Silver' , 'Bronze']].sort_values(by='Gold' , ascending=False).reset_index()
medal_tally['Total'] = medal_tally['Gold'] + medal_tally['Silver'] + medal_tally['Bronze']
medal_tally

,NOC,Team,Gold,Silver,Bronze,Total
0,USA,United States,997,780,682,2459
1,URS,Soviet Union,393,317,294,1004
2,GBR,Great Britain,243,286,276,805
3,GER,Germany,224,256,273,753
4,FRA,France,221,232,260,713
...,...,...,...,...,...,...
1168,USA,United States-14,0,0,0,0
1169,CAN,Beaver,0,0,0,0
1170,USA,United States-3,0,0,1,1
1171,USA,United States-4,0,0,1,1


## Year wise Comparision of Medals by each Country

In [138]:
year_wise_df = df_copy.groupby(['Team', 'Year']).sum()[['Gold', 'Silver', 'Bronze']].sort_values(by=['Year' , 'Gold'],ascending=False).reset_index()
year_wise_df['Total'] = year_wise_df['Gold'] + year_wise_df['Silver'] + year_wise_df['Bronze']
year_wise_df

,Team,Year,Gold,Silver,Bronze,Total
0,United States,2016,45,36,36,117
1,Great Britain,2016,27,23,17,67
2,China,2016,25,18,25,68
3,Russia,2016,18,17,20,55
4,Germany,2016,16,10,15,41
...,...,...,...,...,...,...
4109,Greece-1,1896,0,1,0,1
4110,Greece-2,1896,0,0,0,0
4111,Greece-3,1896,0,0,0,0
4112,Italy,1896,0,0,0,0


### 
Todo:
1) Do Overall analysis of Country 
2) fetch medals on a Particular Year and Country


In [139]:
df_copy.head()

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Bronze,Gold,No Medal,Silver
0,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,No Medal,0,0,1,0
1,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,No Medal,0,0,1,0
2,Gunnar Nielsen Aaby,M,24.0,175.0,71.0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,No Medal,0,0,1,0
3,Edgar Lindenau Aabye,M,34.0,182.0,95.0,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,0,1,0,0
26,"Cornelia ""Cor"" Aalten (-Strannood)",F,18.0,168.0,67.0,Netherlands,NED,1932 Summer,1932,Summer,Los Angeles,Athletics,Athletics Women's 100 metres,No Medal,0,0,1,0


In [140]:
country = df_copy['NOC'].unique().tolist()
country.insert(0 , "Overall")
country

['Overall',
 'CHN',
 'DEN',
 'NED',
 'FIN',
 'NOR',
 'ROU',
 'EST',
 'FRA',
 'MAR',
 'ESP',
 'EGY',
 'IRI',
 'BUL',
 'ITA',
 'CHA',
 'AZE',
 'SUD',
 'RUS',
 'ARG',
 'CUB',
 'BLR',
 'GRE',
 'CMR',
 'TUR',
 'CHI',
 'MEX',
 'USA',
 'URS',
 'NCA',
 'HUN',
 'NGR',
 'ALG',
 'KUW',
 'BRN',
 'PAK',
 'IRQ',
 'UAR',
 'LIB',
 'QAT',
 'MAS',
 'GER',
 'CAN',
 'IRL',
 'AUS',
 'RSA',
 'ERI',
 'TAN',
 'JOR',
 'TUN',
 'LBA',
 'BEL',
 'DJI',
 'PLE',
 'COM',
 'KAZ',
 'BRU',
 'IND',
 'KSA',
 'SYR',
 'MDV',
 'ETH',
 'UAE',
 'YAR',
 'INA',
 'PHI',
 'SGP',
 'UZB',
 'KGZ',
 'TJK',
 'EUN',
 'JPN',
 'CGO',
 'SUI',
 'BRA',
 'GDR',
 'MON',
 'ISR',
 'URU',
 'SWE',
 'SRI',
 'ARM',
 'CIV',
 'KEN',
 'BEN',
 'GBR',
 'GHA',
 'SOM',
 'NIG',
 'MLI',
 'AFG',
 'POL',
 'CRC',
 'PAN',
 'GEO',
 'SLO',
 'GUY',
 'NZL',
 'POR',
 'PAR',
 'ANG',
 'VEN',
 'COL',
 'FRG',
 'BAN',
 'PER',
 'ESA',
 'PUR',
 'UGA',
 'HON',
 'ECU',
 'TKM',
 'MRI',
 'SEY',
 'TCH',
 'LUX',
 'MTN',
 'SKN',
 'TTO',
 'DOM',
 'VIN',
 'JAM',
 'LBR',
 'SUR',
 'NE

In [141]:
years = df_copy['Year'].unique().tolist()
years.sort()
years.insert(0 , "Overall")
# years

In [174]:
medal_df = df_copy[['Team' , 'NOC' , 'Games' ,'Year' , 'City' , 'Sport' , 'Event' , 'No Medal' , 'Gold' , 'Silver' , 'Bronze' , 'Medal']]
# medal_df.isnull().sum()
# medal_df[medal_df['Medal']=='Gold']['NOC'].value_counts()
medal_df

,Team,NOC,Games,Year,City,Sport,Event,No Medal,Gold,Silver,Bronze,Medal
0,China,CHN,1992 Summer,1992,Barcelona,Basketball,Basketball Men's Basketball,1,0,0,0,No Medal
1,China,CHN,2012 Summer,2012,London,Judo,Judo Men's Extra-Lightweight,1,0,0,0,No Medal
2,Denmark,DEN,1920 Summer,1920,Antwerpen,Football,Football Men's Football,1,0,0,0,No Medal
3,Denmark/Sweden,DEN,1900 Summer,1900,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,0,1,0,0,Gold
26,Netherlands,NED,1932 Summer,1932,Los Angeles,Athletics,Athletics Women's 100 metres,1,0,0,0,No Medal
...,...,...,...,...,...,...,...,...,...,...,...,...
271078,Soviet Union,URS,1956 Summer,1956,Melbourne,Athletics,Athletics Women's Shot Put,0,0,1,0,Silver
271080,Soviet Union,URS,1964 Summer,1964,Tokyo,Athletics,Athletics Women's Shot Put,0,0,0,1,Bronze
271087,Poland,POL,1980 Summer,1980,Moskva,Swimming,Swimming Men's 100 metres Butterfly,1,0,0,0,No Medal
271088,Poland,POL,1980 Summer,1980,Moskva,Swimming,Swimming Men's 200 metres Butterfly,1,0,0,0,No Medal


In [ ]:
# medal_df[medal_df['NOC']=='IND'].head()
# medal_df[medal_df['Year'] ==2016].head()
# medal_df[(medal_df['NOC']=='IND') & (medal_df['Year']==2016)]

In [ ]:
# The standard "Overall" Tally
# medal_tally = medal_df.groupby('NOC').sum()['Medal']
medal_tally = df_copy.groupby(['Year']).sum()[['Gold' , 'Silver' , 'Bronze']].sort_values(by='Year' , ascending=True).reset_index()
medal_tally

,Year,Gold,Silver,Bronze
0,1896,43,41,34
1,1900,95,93,92
2,1904,96,92,89
3,1906,74,75,73
4,1908,109,107,106
5,1912,108,104,104
6,1920,158,152,139
7,1924,129,131,131
8,1928,119,118,119
9,1932,125,125,120


## Get Medal Tally Overall of a Country and of a Particular Year

In [194]:
def get_medal_tally(year , country):
    '''
    there can be 4 cases in case of getting medal tally
    1) overall analysis by year of a country
    3) performance by country
    2) performance by year
    4) performance by country and year.
    '''
    flag = 0
    # case1
    if(year == 'Overall' and country =='Overall'):
        temp_df = medal_df
    # case 2
    if(year=='Overall' and country!='Overall'):
       flag = 1
       temp_df =  medal_df[medal_df['NOC']==country] 
    # case 3
    if(year!='Overall' and country =='Overall'):
        temp_df = medal_df[medal_df['Year']==int(year)]
    # case 4
    if( year!='Overall' and country!='Overall'):
       temp_df =  medal_df[(medal_df['NOC']==country) & (medal_df['Year']==year)]
       
    # get medals by each country and Year.
    if(flag==1):
        medal_tally = temp_df.groupby(['Year']).sum()[['Gold' , 'Silver' , 'Bronze']].sort_values(by='Year' , ascending=True).reset_index()
        
    else:
        
        medal_tally = temp_df.groupby(['NOC']).sum()[['Gold' , 'Silver' , 'Bronze']].sort_values(by='Gold' , ascending=False).reset_index()
        
    medal_tally['Total'] = medal_tally['Gold'] + medal_tally['Silver'] + medal_tally['Bronze']
    # print(medal_tally)
    return medal_tally

get_medal_tally(year = 'Overall' , country='IND')

,Year,Gold,Silver,Bronze,Total
0,1900,0,2,0,2
1,1920,0,0,0,0
2,1924,0,0,0,0
3,1928,1,0,0,1
4,1932,1,0,0,1
5,1936,1,0,0,1
6,1948,1,0,0,1
7,1952,1,0,1,2
8,1956,1,0,0,1
9,1960,0,1,0,1
